In [4]:
from urllib import request
from os import path, makedirs

In [6]:
URL = "https://bnlf-tests.s3.eu-central-1.amazonaws.com/recipes.json"
FOLDER = '../data'
LOCAL_FILE = f"{FOLDER}/recipes.json"

if not path.exists(FOLDER):
    makedirs(FOLDER)

request.urlretrieve(URL, LOCAL_FILE)

('../data/recipes.json', <http.client.HTTPMessage at 0x1076c3c10>)

In [15]:
import polars as pl

pl.Config.set_fmt_str_lengths(300)

polars.config.Config

In [16]:
df = pl.read_ndjson(LOCAL_FILE).drop(['url', 'image'])
df.head()

name,ingredients,cookTime,recipeYield,datePublished,prepTime,description
str,str,str,str,str,str,str
"""Easter Leftover Sandwich""","""12 whole Hard Boiled Eggs 1/2 cup Mayonnaise 3 Tablespoons Grainy Dijon Mustard Salt And Pepper, to taste Several Dashes Worcestershire Sauce Leftover Baked Ham, Sliced Kaiser Rolls Or Other Bread Extra Mayonnaise And Dijon, For Spreading Swiss Cheese Or Other Cheese Slices Thinly Sliced Red…","""PT""","""8""","""2013-04-01""","""PT15M""","""Got leftover Easter eggs? Got leftover Easter ham? Got a hearty appetite? Good! You've come to the right place! I..."""
"""Pasta with Pesto Cream Sauce""","""3/4 cups Fresh Basil Leaves 1/2 cup Grated Parmesan Cheese 3 Tablespoons Pine Nuts 2 cloves Garlic, Peeled Salt And Pepper, to taste 1/3 cup Extra Virgin Olive Oil 1/2 cup Heavy Cream 2 Tablespoons Butter 1/4 cup Grated Parmesan (additional) 12 ounces, weight Pasta (cavitappi, Fusili, Etc.) 2 whol…","""PT10M""","""8""","""2011-06-06""","""PT6M""","""I finally have basil in my garden. Basil I can use. This is a huge development. I had no basil during the winter. None. G..."""
"""Herb Roasted Pork Tenderloin with Preserves""","""2 whole Pork Tenderloins Salt And Pepper, to taste 8 Tablespoons Herbs De Provence (more If Needed 1 cup Preserves (fig, Peach, Plum) 1 cup Water 1 Tablespoon Vinegar""","""PT15M""","""12""","""2011-09-15""","""PT5M""","""This was yummy. And easy. And pretty! And it took basically no time to make. Before I share the recipe, I'll just say it:..."""
"""Chicken Florentine Pasta""","""1 pound Penne 4 whole Boneless, Skinless Chicken Breasts Salt And Pepper, to taste 2 Tablespoons Butter 2 Tablespoons Olive Oil 4 cloves Garlic, Minced 3/4 cups Dry White Wine 3/4 cups Low-sodium Broth, More If Needed 1 bag Baby Spinach 2 cups Grape Tomatoes, Halved Lengthwise 4 ounces, weight Par…","""PT20M""","""10""","""2012-04-23""","""PT10M""","""I made this for a late lunch Saturday, and it absolutely completed me. It also absolutely completed my fourteen-year-old daug..."""
"""Perfect Iced Coffee""","""1 pound Ground Coffee (good, Rich Roast) 8 quarts Cold Water Half-and-half (healthy Splash Per Serving) Sweetened Condensed Milk (2-3 Tablespoons Per Serving) Note: Can Use Skim Milk, 2% Milk, Whole Milk, Sugar, Artificial Sweeteners, Syrups...adapt To Your Liking!""","""PT""","""24""","""2011-06-13""","""PT8H""","""Iced coffee is my life. When I wake up, often around the time party animals on the west coast are just heading home, I start ..."""


In [14]:
df.estimated_size(unit='mb')

0.6133699417114258